In [2]:
from datascience import *

Table.read_table('film/Arizona Defense.csv').where('pff_RUNPASS', 'P').select('#', 'pff_PENALTY', 'pff_SACK', 'pff_PASSROUTETARGET', 'pff_PASSPATTERNBASIC', 'pff_PASSPATTERNBYPLAYER', 'pff_PASSRESULT')


#,pff_PENALTY,pff_SACK,pff_PASSROUTETARGET,pff_PASSPATTERNBASIC,pff_PASSPATTERNBYPLAYER,pff_PASSRESULT
1,nan,nan,0(-2),0; 9; 6; 9; 0,ATL 40 0(0); ATL 17 9z; ATL 46 26(5); ATL 05 9z; ATL 84 ...,COMPLETE
6,nan,nan,6(5),6; 3; H2; 6; 6,ATL 12 6(5); ATL 85 1; ATL 25 h2(-8); ATL 35 6o(8); ATL ...,COMPLETE
8,nan,nan,k1,X; X; H5; 3; 7,ATL 05 xo(11); ATL 80 xg(-3); ATL 35 flh31(-5); ATL 85 k ...,COMPLETE
9,nan,nan,nan,4; -; H8; X; 3,ATL 17 4(14); ATL 85 k; ATL 25 frh8(-3); ATL 46 klxu(-3) ...,RUN
11,nan,nan,flh1(-3),X; X; H1; 3; x7,ATL 12 xo(17); ATL 85 krxu(-5); ATL 84 flh1(-3); ATL 80 ...,COMPLETE
12,nan,nan,frh2(-4),-; H2; X; 4; 9,ATL 46 k; ATL 84 frh2(-4); ATL 40 klxu(-3); ATL 05 4o(17 ...,COMPLETE
14,nan,nan,4(11),X; 4; X; H3; 4,ATL 14 xo(22); ATL 17 4o(16); ATL 46 krxu(-3); ATL 25 fl ...,INCOMPLETE
16,nan,nan,6o(6),6; 2; 3; H4; 6,ATL 17 6(6); ATL 84 2(6); ATL 85 3(6); ATL 25 krh21(1); ...,COMPLETE
21,nan,nan,3(4),3; 3; H2; 4; 6,ATL 14 3(15); ATL 05 3(4); ATL 84 krh2(-7); ATL 46 4(14) ...,COMPLETE
23,nan,nan,frh21(-3),-; H4; X; 3; 7,ATL 85 k; ATL 25 frh21(-3); ATL 46 klxu(-3); ATL 05 3(20 ...,COMPLETE


In [3]:
"""VideoHandler Module"""
import cv2 as cv
import numpy as np
import os


class VideoHandler:
    video_player: cv.VideoCapture
    currentFrame: np.ndarray
    frame_number: int = 0
    isPlaying: bool = True


    def __init__(self, videoPath: str = "film/22DP BUF OFF vs CIN 01-22-23"):
        if not os.path.exists(f"{videoPath}.mp4"):
            raise Exception(f"Video path {videoPath}.mp4 not found")
        self.video_player = cv.VideoCapture(f"{videoPath}.mp4")
        if not self.video_player.isOpened():
            raise Exception("Error: Could not open video file")
        self.feed()

    
    def startFeed(self):
        self.isPlaying = True

    def pauseFeed(self):
        self.isPlaying = False
    
    def feed(self) -> int:
        if self.isPlaying:
            # Capture frame-by-frame
            ret, frame = self.video_player.read()
            # if frame is read correctly ret is True
            if not ret:
                print("Can't receive frame (stream end?). Exiting ...")
                return
            self.frame_number += 1
            self.currentFrame = frame
        return self.frame_number

            
    def skip_to_frame(self, frame_number: int):
        if frame_number < 0 or frame_number > self.video_player.get(cv.CAP_PROP_FRAME_COUNT):
            print(frame_number)
            print(self.video_player.get(cv.CAP_PROP_FRAME_COUNT))
            raise Exception("Error: Frame Index Out of Bounds")
        self.video_player.set(cv.CAP_PROP_POS_FRAMES, frame_number)
        self.frame_number = frame_number
        if not self.isPlaying:
            self.isPlaying = True
            self.feed()
            self.isPlaying = False
            return
        self.feed()
    
    def set_video(self, path: str):
        if self.video_player.isOpened():
            self.video_player.release()
        if not path.endswith(".mp4"):
            raise Exception("Error: Invalid Data Type for video")
        self.video_player = cv.VideoCapture(path)
        if not self.video_player.isOpened():
            raise Exception("Error: Could not open video file")
        self.skip_to_frame(0)
        self.feed()
    
    def getCurrentFrame(self) -> np.ndarray:
        """Retrieves current frame"""
        cf = self.currentFrame
        return cf










    
        

In [4]:
import cv2 as cv
def new_clip(play_number, v, cam=None):
        """Function that ends current clip and begins a new one"""
        if  cam != None:
            cam.release()

        #Create new clip for next view
        return cv.VideoWriter(
            f"split_clips/buf/BUF vs CIN {play_number}.mp4",
            cv.VideoWriter_fourcc(*'mp4v'),
            60,
            (v.currentFrame.shape[1], v.currentFrame.shape[0])
            )

def add_frame(cam, frame: np.array):
        """Function that adds a given frame to the clip"""
        try:
            cam.write(frame)
        except:
            print("Video not open. Done recording clips?")

In [8]:
import xml.etree.ElementTree as ET
full_game_tree = ET.parse("film/22DP BUF OFF vs CIN 01-22-23.xchange")

clips = [x.text for x in full_game_tree.findall("Plays/Play/Views/View/MarkIn")]
durs = [x.text for x in full_game_tree.findall("Plays/Play/Views/View/Duration")]
len(durs)


64

In [2]:
v= VideoHandler()
first = clips.pop(0)
while v.frame_number < int(first):
    v.feed()
clip_num = 1
cam = None
for outmark in clips:
    cam: cv.VideoWriter = new_clip(clip_num, v, cam)
    while v.frame_number < int(outmark):
        add_frame(cam, v.currentFrame)
        v.feed()
    clip_num += 1

cam: cv.VideoWriter = new_clip(clip_num, v, cam)
while v.feed():
    add_frame(cam,v.currentFrame)
cam.release()
        

KeyboardInterrupt: 